In [1]:
import sys
import numpy as np
import pandas as pd
import os
import matplotlib as plt
sys.path.append('..')
print(sys.path)
from nlp_pipeline.feature_extraction import FeatureExtractor
from nlp_pipeline.preprocess_text import TextPreprocessor
from model.evaluate_model import ModelEvaluator
from model.train_baseline import BaselineTrainer
from interpretation.lime_interpreter import LimeTextInterpreter
from interpretation.shap_interpreter import ShapInterpreter
from nlp_pipeline.back_translator import BackTranslationAugmentor
from nlp_pipeline.embedding_oversample import EmbeddingOversampler

['d:\\aco\\research\\Asd-classification\\notebook', 'c:\\Users\\MufliDevs\\anaconda3\\python312.zip', 'c:\\Users\\MufliDevs\\anaconda3\\DLLs', 'c:\\Users\\MufliDevs\\anaconda3\\Lib', 'c:\\Users\\MufliDevs\\anaconda3', '', 'c:\\Users\\MufliDevs\\anaconda3\\Lib\\site-packages', 'c:\\Users\\MufliDevs\\anaconda3\\Lib\\site-packages\\win32', 'c:\\Users\\MufliDevs\\anaconda3\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\MufliDevs\\anaconda3\\Lib\\site-packages\\Pythonwin', 'c:\\Users\\MufliDevs\\anaconda3\\Lib\\site-packages\\setuptools\\_vendor', '..']


Load dataset + Augmented

In [ ]:
df = df = pd.read_csv('../data/feature/merged_final.csv', encoding='latin1')
columns = ['label', 'transcription', 'total_words', 'unique_words', 'num_sentences',
                   'stopwords', 'num_adjectives', 'num_nouns', 'num_verbs', 'num_adverbs',
                   'type_token_ratio', 'avg_words_per_sentence']
df = df[columns]
augmentor = BackTranslationAugmentor()
df_augmented = augmentor.augment_dataframe(df)
print(df_augmented.head)
df_augmented.to_csv('../data/feature/1_augmented.csv', index=False)

Preprocessing text

In [2]:
df = df = pd.read_csv('../data/feature/combined_augmented_asd.csv', encoding='latin1')
text_preprocessor = TextPreprocessor()
df['clean_text'] = df['transcription'].apply(text_preprocessor.preprocess)
df[['transcription', 'clean_text']].head()

,transcription,clean_text
0,Tunjuk kaka coba,tunjuk kaka coba
1,Inii!,ini
2,Siapa namamu?,siapa nama
3,Iyaaaa?,
4,Kenalan duluu!,kenal duluu


In [ ]:
df['label'].value_counts()
df['label'].value_counts()

Ekstraksi fitur with TFIDF

In [ ]:
extractor = FeatureExtractor()
X = extractor.fit_transform_tfidf(df['clean_text'])
y = df['label'].values
print(X.shape, y.shape)

Oversampling with TFIDF

In [ ]:
X_dense = extractor.fit_transform_tfidf(df['clean_text']).toarray()

ling_cols = extractor.linguistic_cols
X_ling = df[ling_cols].values

y = (df['label'].str.upper().str.strip() == 'ASD').astype(int).values

oversampler = EmbeddingOversampler()
X_embed_bal, X_ling_bal, y_bal = oversampler.oversample(X_dense, X_ling, y)


Ekstraksi fitur with IndoBERT

In [ ]:
extractor = FeatureExtractor()
X = extractor.encode_series_bert(df['clean_text'])
y = (df['label'].str.upper().str.strip() == 'ASD').astype(int).values
ling_cols = extractor.linguistic_cols
X_ling = df[ling_cols].values
oversampler = EmbeddingOversampler()
X_embed_bal, X_ling_bal, y_bal = oversampler.oversample(X, X_ling, y)

print(X_embed_bal.shape, y.shape)

SVC WITH 5 FOLD

In [ ]:
trainer = BaselineTrainer()

evaluator = ModelEvaluator(model_type='svm',pos_label=1)

results = evaluator.cross_validate_oversample_with_confusionmatrix(X_embed_bal,y_bal)

USE THIS IF YOU NOT USING BERT

In [ ]:
final_model = trainer.train(X_embed_bal, y_bal)

lime_interpreter = LimeTextInterpreter(
    model=final_model,
    vectorizer=extractor.get_tfidf_vectorizer(),
    class_names=['NON ASD', 'ASD']
)

asd_samples = df[df['label'] == 'ASD'].sample(n=12, random_state=42)
non_asd_samples = df[df['label'] == 'NON ASD'].sample(n=12, random_state=42)
lime_samples = pd.concat([asd_samples, non_asd_samples]).reset_index(drop=True)

lime_interpreter.save_lime_explanation_to_csv(lime_samples,"../reports/lime_results.csv")

In [ ]:
final_model = BaselineTrainer('svm').train(X_embed_bal,y_bal)
interpreter = ShapInterpreter(model=final_model)
sampled_df = df.sample(n=100, random_state=42)
texts = sampled_df['clean_text'].tolist()
labels = sampled_df['label'].tolist()

shap_values = interpreter.explain(texts)
interpreter.save_shap_explanations_to_csv(
    shap_values=shap_values,
    texts=texts,
    output_path="../reports/shap_results_SVM_oversample+bert.csv",
    true_labels=labels
)

LINEAR REGRETION WITH 5 FOLD

In [ ]:
trainer = BaselineTrainer()
evaluator = ModelEvaluator(pos_label=1)

results = evaluator.cross_validate_oversample_with_confusionmatrix(X_embed_bal,y_bal)

In [ ]:
final_model = trainer.train(X, y)

lime_interpreter = LimeTextInterpreter(
    model=final_model,
    vectorizer=extractor.get_tfidf_vectorizer(),
    class_names=['NON ASD', 'ASD']
)

asd_samples = df[df['label'] == 'ASD'].sample(n=12, random_state=42)
non_asd_samples = df[df['label'] == 'NON ASD'].sample(n=12, random_state=42)
lime_samples = pd.concat([asd_samples, non_asd_samples]).reset_index(drop=True)

lime_interpreter.save_lime_explanation_to_csv(lime_samples,"../reports/lime_results_LOGRES.csv")

SVM WITH 10 FOLD

In [4]:
trainer = BaselineTrainer()
evaluator = ModelEvaluator(model_type='svm', n_splits= 10, random_state=32,pos_label=1)

results = evaluator.cross_validate_oversample_with_confusionmatrix(X_embed_bal,y_bal)


[Oversampled] Fold 1


KeyboardInterrupt: 

In [ ]:
final_model = trainer.train(X, y)

lime_interpreter = LimeTextInterpreter(
    model=final_model,
    vectorizer=extractor.get_tfidf_vectorizer(),
    class_names=['NON ASD', 'ASD']
)

asd_samples = df[df['label'] == 'ASD'].sample(n=12, random_state=42)
non_asd_samples = df[df['label'] == 'NON ASD'].sample(n=12, random_state=42)
lime_samples = pd.concat([asd_samples, non_asd_samples]).reset_index(drop=True)

lime_interpreter.save_lime_explanation_to_csv(lime_samples,"../reports/lime_results_SVM10FOLD.csv")

In [ ]:
final_model = BaselineTrainer('svm').train(X,y)
interpreter = ShapInterpreter(model=final_model)
sampled_df = df.sample(n=100, random_state=42)
texts = sampled_df['clean_text'].tolist()
labels = sampled_df['label'].tolist()

shap_values = interpreter.explain(texts)
interpreter.save_shap_explanations_to_csv(
    shap_values=shap_values,
    texts=texts,
    output_path="../reports/shap_results_SVM_10_fold.csv",
    true_labels=labels
)

LINEAR REGRETION WITH 10 FOLD

In [ ]:
trainer = BaselineTrainer()
evaluator = ModelEvaluator(n_splits= 10, random_state=32,pos_label=1)

results = evaluator.cross_validate_oversample_with_confusionmatrix(X_embed_bal,y_bal)

In [ ]:
final_model = BaselineTrainer('svm').train(X,y)
interpreter = ShapInterpreter(model=final_model)
sampled_df = df.sample(n=100, random_state=42)
texts = sampled_df['clean_text'].tolist()
labels = sampled_df['label'].tolist()

shap_values = interpreter.explain(texts)
interpreter.save_shap_explanations_to_csv(
    shap_values=shap_values,
    texts=texts,
    output_path="../reports/shap_results_Logres_10_fold.csv",
    true_labels=labels
)


In [ ]:
interpreter.visualize(shap_values= shap_values)

FUSION

In [3]:
extractor = FeatureExtractor()
X = extractor.extract_fused_features_bert(df)
y = (df['label'].str.upper().str.strip() == 'ASD').astype(int).values
ling_cols = extractor.linguistic_cols
X_ling = df[ling_cols].values
oversampler = EmbeddingOversampler()
X_embed_bal, X_ling_bal, y_bal = oversampler.oversample(X, X_ling, y)

print(X_embed_bal.shape, y.shape)

tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/229k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

(9296, 778) (8380,)


Fusion with tfidf

In [ ]:
extractor = FeatureExtractor()
X = extractor.extract_fused_features_tfidf(df, fit=True)
y = (df['label'].str.upper().str.strip() == 'ASD').astype(int).values
ling_cols = extractor.linguistic_cols
X_ling = df[ling_cols].values
oversampler = EmbeddingOversampler()
X_embed_bal, X_ling_bal, y_bal = oversampler.oversample(X, X_ling, y)

print(X_embed_bal.shape, y.shape)

In [ ]:
evaluator = ModelEvaluator(model_type='svm', n_splits= 10, random_state=32)
results = evaluator.cross_validate_with_confusionmatrix(X, y)


Testing Model

In [ ]:
trainer = BaselineTrainer(model_type='svm')
model = trainer.train(X_embed_bal, y_bal)

new_text = "Apakah kamu suddah makan?"
clean_text = text_preprocessor.preprocess(new_text)
X_new = extractor.encode_series_bert([clean_text])
predicted_label = model.predict(X_new)[0]
print(f"Predicted label: {predicted_label}")

In [ ]:
new_text = "Apakah kamu sudah makan?"
clean_text = text_preprocessor.preprocess(new_text)
X_new = extractor.encode_series_bert([clean_text])
predicted_label = model.predict(X_new)[0]
print(f"Predicted label: {predicted_label}")